# Data and Configuration

This notebook provides examples of data pipelines, data space building, and configuration managment, in addition to examples of model components being used (see section below).
The following examples are provided:
* Baseline parameter confiugration
* Data space

In [2]:
from giga.app.config_client import ConfigClient
from giga.app.config import get_country_default

# Let's load in the global app configuration
workspace = 'workspace' # the workspace where data for each country of interest can be found
country = 'RWA'

global_config = ConfigClient.from_country_defaults(
                    get_country_default(country, workspace=workspace)
                )

# Component Drivers

This notebook provides examples of component drivers that show how to configure, and run a model component.
The following are shown:

* Fiber Model
* Satellite Model
* Cellular Model


## Data Space

## Fiber Component Model

In [ ]:
from giga.models.components.fiber_cost_model import FiberCostModel
from giga.schemas.conf.models import FiberTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = FiberTechnologyCostConf(
            capex={
                "cost_per_km": 7_500, # USD
                "economies_of_scale": True,
            },
            opex={
                "cost_per_km": 100, # USD
                "annual_bandwidth_cost_per_mbps": 10, # in USD
            },
            constraints={
                "maximum_connection_length": 20_000, # in meters
                "required_power": 500, # in kWh
                "maximum_bandwithd": 2_000.0, # mbps
            },
            electricity_config={"capex": {"solar_cost_per_watt": 500.0},
                                "opex": {"cost_per_kwh": 0.1},
                                "constraints":{"required_power_per_school": 11000}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = FiberCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Fiber' for r in outputs.cost_results])
print(outputs.cost_results[0].feasible) 
print(outputs.cost_results[1].feasible)

## Sattelite Component Model

In [3]:
from giga.models.components.satellite_cost_model import SatelliteCostModel
from giga.schemas.conf.models import SatelliteTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = SatelliteTechnologyCostConf(
            capex={
                "fixed_costs": 500 # USD hardware installation
            },
            opex={
                "fixed_costs": 0.0, # USD hardware maintance
                "annual_bandwidth_cost_per_mbps": 15.0,
            },
            constraints={
                "maximum_bandwithd": 150.0,  # should be pulled from defaults
                "required_power": 200.0,
            },
            electricity_config={"capex": {"solar_cost_per_watt": 500.0},
                                "opex": {"cost_per_kwh": 0.1},
                                "constraints":{"required_power_per_school": 11000}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = SatelliteCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Satellite' for r in outputs.cost_results])
assert outputs.cost_results[0].capex == 500.0 # Fixed install cost
assert outputs.cost_results[0].opex == 300.0 # Operating cost

INFO       2023-11-09 07:40:45,505       : Starting Satellite Cost Model


## Cellular Component Model

In [8]:
from giga.models.components.cellular_cost_model import CellularCostModel
from giga.schemas.conf.models import CellularTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = CellularTechnologyCostConf(
            capex={"fixed_costs": 500.0},
            opex={
                "fixed_costs": 0.0,
                "annual_bandwidth_cost_per_mbps": 10.0,
            },
            constraints={"maximum_bandwithd": 100.0,
                         "required_power": 10.0,
                         "maximum_range": 8_000}, # in m
            electricity_config={"capex": {"solar_cost_per_watt": 500.0},
                                "opex": {"cost_per_kwh": 0.1},
                                "constraints":{"required_power_per_school": 11000}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = CellularCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Cellular' for r in outputs.cost_results])
print(outputs.cost_results[10].feasible)
print(outputs.cost_results[100].feasible)

INFO       2023-11-09 07:42:45,778       : Starting Cellular Cost Model


False
False


## P2P Component Model

In [9]:
from giga.models.components.p2p_cost_model import P2PCostModel
from giga.schemas.conf.models import P2PTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = P2PTechnologyCostConf(
            capex={"fixed_costs": 500.0},
            opex={
                "fixed_costs": 0.0,
                "annual_bandwidth_cost_per_mbps": 10.0,
            },
            constraints={"maximum_bandwithd": 100.0,
                         "required_power": 10.0,
                         "maximum_range": 8_000}, # in m
            electricity_config={"capex": {"solar_cost_per_watt": 500.0},
                                "opex": {"cost_per_kwh": 0.1},
                                "constraints":{"required_power_per_school": 11000}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = P2PCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'P2P' for r in outputs.cost_results])
assert outputs.cost_results[0].capex == 500.0 # Fixed install cost
assert outputs.cost_results[0].opex == 200.0 # Operating cost

INFO       2023-11-09 07:48:25,111       : Starting P2P Cost Model
